In [1]:
import pandas as pd
import numpy as np

In [37]:
emb_df = pd.read_csv('/itf-fi-ml/shared/users/ziyuzh/svm/data/pre_processed_features/expression_emb/SCGPT-HUMAN/SCGPT-HUMAN_emb.csv',header = None)
emb_df.head(2)

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
0,-0.021478,-0.056237,0.049937,0.062038,-0.047067,-0.031588,0.018498,-0.158697,0.129990,-0.070728,...,-0.078208,0.031348,-0.030603,0.053489,0.033255,0.004434,0.060634,0.040438,-0.020253,0.055903
1,0.047382,-0.084310,0.049981,0.063209,0.062351,0.019465,0.005474,0.014955,-0.008151,-0.040076,...,-0.082427,0.070376,-0.058187,-0.113961,-0.025331,-0.007994,0.099673,-0.059459,-0.049220,-0.026471


In [4]:
gene_list = pd.read_csv('/itf-fi-ml/shared/users/ziyuzh/svm/data/pre_processed_features/expression_emb/SCGPT-HUMAN/SCGPT-HUMAN_genelist.txt',header = None)
gene_list.head(2)

,0
0,1
1,10


In [12]:
import mygene

mg = mygene.MyGeneInfo()

In [27]:
map_df = mg.getgenes(gene_list[0], fields='name,symbol,entrezgene,taxid',as_dataframe=True)

Input sequence provided is already in string format. No operation performed


In [28]:
map_df

,_id,_version,entrezgene,name,symbol,taxid
query,,,,,,
1,1,1,1,alpha-1-B glycoprotein,A1BG,9606
10,10,1,10,N-acetyltransferase 2,NAT2,9606
100,100,1,100,adenosine deaminase,ADA,9606
1000,1000,2,1000,cadherin 2,CDH2,9606
10000,10000,2,10000,AKT serine/threonine kinase 3,AKT3,9606
...,...,...,...,...,...,...
9987,9987,2,9987,heterogeneous nuclear ribonucleoprotein D like,HNRNPDL,9606
9988,9988,1,9988,cyclin D binding myb like transcription factor 1,DMTF1,9606
999,999,2,999,cadherin 1,CDH1,9606


In [30]:
import os
local_stringdb = '/itf-fi-ml/shared/users/ziyuzh/svm/data/stringdb/2023'

ppidf = pd.read_csv(os.path.join(local_stringdb,'9606.protein.info.v12.0.txt'), sep='\t', header=0, usecols=['#string_protein_id', 'preferred_name'])
ppidf['preferred_name'] = ppidf['preferred_name'].str.upper()
stringId2name = ppidf.set_index('#string_protein_id')['preferred_name'].to_dict()
name2stringId = ppidf.set_index('preferred_name')['#string_protein_id'].to_dict()
ppidf = pd.read_csv(os.path.join(local_stringdb,'9606.protein.aliases.v12.0.txt'), sep='\t', header=0, usecols=['#string_protein_id', 'alias']).drop_duplicates(['alias'], keep='first')
ppidf['alias'] = ppidf['alias'].str.upper()
aliases2stringId = ppidf.set_index('alias')['#string_protein_id'].to_dict()

def string_convert(gene):
    if gene in name2stringId.keys():
        return name2stringId[gene]
    elif gene in aliases2stringId.keys():
        return aliases2stringId[gene]
    else:
        return None

In [31]:
entrz2string = dict()
for gene in map_df['symbol']:
    entrz2string[gene] = string_convert(gene)

In [32]:
map_df.columns

Index(['_id', '_version', 'entrezgene', 'name', 'symbol', 'taxid'], dtype='object')

In [33]:
map_id = map_df[['_id','symbol']]

In [34]:
map_id['string_id'] = map_id['symbol'].map(entrz2string)

/tmp/ipykernel_3798704/1145848674.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  map_id['string_id'] = map_id['symbol'].map(entrz2string)


In [43]:
map_id = map_id.reset_index()

In [45]:
df_combined = emb_df.loc[map_id[~map_id['string_id'].isna()].index]

In [46]:
df_combined['string_id'] = map_id[~map_id['string_id'].isna()]['string_id']

In [47]:
new_columns = ['string_id'] + [f'feature_{i}' for i, col in enumerate(df_combined.columns) if col != 'string_id']

# Reorder the DataFrame so that 'string_id' is the first column
df_combined = df_combined[['string_id'] + [col for col in df_combined.columns if col != 'string_id']]
df_combined.columns = new_columns

df_combined.head(2)

,string_id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_502,feature_503,feature_504,feature_505,feature_506,feature_507,feature_508,feature_509,feature_510,feature_511
0,9606.ENSP00000263100,-0.021478,-0.056237,0.049937,0.062038,-0.047067,-0.031588,0.018498,-0.158697,0.129990,...,-0.078208,0.031348,-0.030603,0.053489,0.033255,0.004434,0.060634,0.040438,-0.020253,0.055903
1,9606.ENSP00000286479,0.047382,-0.084310,0.049981,0.063209,0.062351,0.019465,0.005474,0.014955,-0.008151,...,-0.082427,0.070376,-0.058187,-0.113961,-0.025331,-0.007994,0.099673,-0.059459,-0.049220,-0.026471


In [49]:
df_combined.to_csv('/itf-fi-ml/shared/users/ziyuzh/svm/data/pre_processed_features/expression_emb/SCGPT-HUMAN/scgpt.csv',index=False)

In [50]:
df_combined

,string_id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_502,feature_503,feature_504,feature_505,feature_506,feature_507,feature_508,feature_509,feature_510,feature_511
0,9606.ENSP00000263100,-0.021478,-0.056237,0.049937,0.062038,-0.047067,-0.031588,0.018498,-0.158697,0.129990,...,-0.078208,0.031348,-0.030603,0.053489,0.033255,0.004434,0.060634,0.040438,-0.020253,0.055903
1,9606.ENSP00000286479,0.047382,-0.084310,0.049981,0.063209,0.062351,0.019465,0.005474,0.014955,-0.008151,...,-0.082427,0.070376,-0.058187,-0.113961,-0.025331,-0.007994,0.099673,-0.059459,-0.049220,-0.026471
2,9606.ENSP00000361965,-0.000216,-0.070240,0.023673,0.009852,-0.033579,-0.083926,0.061259,-0.028196,0.018252,...,-0.021560,0.017004,-0.023226,-0.027589,0.031865,0.147832,-0.075468,-0.004255,0.077796,0.002969
3,9606.ENSP00000269141,0.039682,0.032027,-0.004452,0.040624,0.116986,0.015869,-0.073001,0.145058,-0.003429,...,0.139090,-0.008733,0.046119,-0.011149,0.007122,0.019908,-0.005020,-0.045655,0.089971,-0.090069
4,9606.ENSP00000500582,0.077371,0.166191,-0.083184,0.110919,0.046064,0.102961,-0.062631,-0.058595,0.057053,...,0.174290,-0.024523,0.093618,0.067001,0.065056,-0.024112,-0.118960,-0.026028,0.022981,0.072078
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11350,9606.ENSP00000483254,0.052945,0.121622,0.006746,-0.035257,-0.082598,-0.016596,0.041922,-0.079131,-0.072259,...,-0.022651,0.043252,-0.004944,0.095187,-0.019664,-0.039506,-0.010746,0.066422,0.043152,-0.029366
11351,9606.ENSP00000378193,0.070723,0.090786,0.069816,0.058018,-0.009318,0.021303,0.079645,0.056234,0.060629,...,-0.000845,-0.008505,0.010234,-0.041829,0.005612,0.004393,0.042982,0.027303,0.056942,-0.052819
11352,9606.ENSP00000261769,-0.073119,-0.043803,0.014063,0.001114,-0.012400,0.130925,-0.070928,0.003086,-0.014011,...,0.067835,0.070784,-0.009906,0.020045,-0.073636,0.010186,0.059835,-0.006438,-0.061370,-0.016086
11353,9606.ENSP00000346112,0.156123,0.056467,-0.036105,0.008067,-0.025250,-0.016168,0.010718,-0.049104,-0.073923,...,-0.064242,-0.035390,-0.140214,0.165520,-0.028362,-0.044531,-0.043150,0.113256,0.075213,-0.104206
